**Проверка версии:**

In [1]:
import sqlalchemy
sqlalchemy.__version__

'2.0.19'

Любое приложение SQLAlchemy начинается с создания движка ([объекта Engine](https://docs.sqlalchemy.org/en/20/tutorial/engine.html#establishing-connectivity-the-engine)).

In [2]:
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://w9i@localhost/w9i", echo=True)

Функция `create_engine` возвращает нам объект `Engine`.

Объект `Engine` в свою очередь предоставляет нам единицу соединения с БД, называемую [`Connection`](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Connection).

Объект `Connection` *приобретается* путем вызова `Engine.connect()`.

`Connection` предоставляет нам возможность выполнять SQL-запросы, а также управлять транзакциями.

Объект `Connection` представляет собой одно DBAPI-соединение, извлеченное из пула соединений. Для правильного управления пулом соединений необходимо возвращать соединения в пул (т.е. `Connection.close()`) каждый раз, когда соединение не используется.

Поэтому мы будем создавать соединение в контекст менеджере.

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(
        text("SELECT 'Hello, SQLAlchemy World!'")
    )
    print(result.all())

2023-07-17 21:36:57,452 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-07-17 21:36:57,453 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 21:36:57,454 INFO sqlalchemy.engine.Engine select current_schema()
2023-07-17 21:36:57,454 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 21:36:57,454 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-07-17 21:36:57,454 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-07-17 21:36:57,455 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,456 INFO sqlalchemy.engine.Engine SELECT 'Hello, SQLAlchemy World!'
2023-07-17 21:36:57,456 INFO sqlalchemy.engine.Engine [generated in 0.00059s] {}
[('Hello, SQLAlchemy World!',)]
2023-07-17 21:36:57,456 INFO sqlalchemy.engine.Engine ROLLBACK


Импортированная нами функция `text`, предоставляет нам возможность безопасно писать запросы в *стиле* `SQL`. Эта функция будет более подробно рассмотрена позже.

Результат нашего `SELECT` был возвращен в объекте [Result](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Result), который будет рассмотрен позже, но пока отметим, что этот объект должен использоваться внутри блока "connect" и не передаватся за пределы области действия нашего соединения (контекст менеджера).

Из лога, который выдает нам SQLAlchemy, благодаря флагу `echo=True`, мы можем увидеть что наши запросы **неявно** являются транзакционными, но *незакомиченными* (то есть незафиксированная транзакция).

Коммитить изменения мы должны явно, используя `.commit()` на объекте `Connection`.

Давайте создадим таблицу, внесем какие-то данные и закоммитим изменения.

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))  # Создаем таблицу
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),  # Вставка данных в таблицу
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}],                   # (параметризированный запрос).
    )
    conn.commit()  # Коммит изменений.

2023-07-17 21:36:57,460 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,461 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-07-17 21:36:57,461 INFO sqlalchemy.engine.Engine [generated in 0.00080s] {}
2023-07-17 21:36:57,463 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 21:36:57,464 INFO sqlalchemy.engine.Engine [generated in 0.00029s] [{'x': 1, 'y': 1}, {'x': 2, 'y': 4}]
2023-07-17 21:36:57,464 INFO sqlalchemy.engine.Engine COMMIT


Выше мы выполнили два SQL-оператора, которые являются траназкционными.

1. Оператор `CREATE TABLE`
2. Параметризированный оператор `INSERT`

Синтаксис параметризации, будет рассмотрен ниже.

Мы также закоммитили наши изменения, это означает что сейчас в БД у нас таблица `some-table` с внесенными в нее нами данными.

Мы также можем продолжать писать другие запросы, и также вызывать `Connection.commit()` для последующих SQL-операторов.

Существует другой способ фиксации данных, для этого мы можем заранее объявить наш блок connect блоком транзакций.
В этом случае мы используем `Engine.begin()` а не `Engine.connect()`.

In [5]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}],
    )

2023-07-17 21:36:57,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,469 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 21:36:57,469 INFO sqlalchemy.engine.Engine [cached since 0.005555s ago] [{'x': 6, 'y': 8}, {'x': 9, 'y': 10}]
2023-07-17 21:36:57,470 INFO sqlalchemy.engine.Engine COMMIT


Чтобы получить резульирующие строки, мы воспользуемся SQL-оператором `SELECT`.

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-07-17 21:36:57,474 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,474 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2023-07-17 21:36:57,474 INFO sqlalchemy.engine.Engine [generated in 0.00073s] {}
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-07-17 21:36:57,475 INFO sqlalchemy.engine.Engine ROLLBACK


Выше, были выбраны все строки из нашей таблицы. 

Возвращаемый объект называется `Result` и представляет собой итерируемый объект резульирующих строк.

`Result` имеет множество методов для получения и преобразования резульирующих строк, например, метод `Result.all()`, показанный ранее, который возвращает список всех объектов [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row). Кроме того, в нем реализован интерфейс итератора Python, что позволяет выполнять итерации непосредственно над коллекцией объектов [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row).

Сами объекты [Row](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.Row) похожи на [именованные кортежи](https://docs.python.org/3/library/collections.html#collections.namedtuple) Python. 

Ниже мы проиллюстрируем различные способы доступа к резульирующим строкам.

```python
# Tuple Assignment :)
result = conn.execute(text("select x, y from some_table"))

for x, y in result:
    ...


# извлечение по индексу
result = conn.execute(text("select x, y from some_table"))

for row in result:
    x = row[0]


# извлечение по атрибуту
result = conn.execute(text("select x, y from some_table"))

for row in result:
    y = row.y

    print(f"Row: {row.x} {y}")


# Mapping Access
result = conn.execute(text("select x, y from some_table"))

for dict_row in result.mappings():
    x = dict_row["x"]
    y = dict_row["y"]
```

Наконец-то параметризированные запросы.

В наших SQL запросах мы каким-то образом должны передавать данные. 
`Connection.execute()` умеет принимать параметры, которые называются [связанные параметры](https://docs.sqlalchemy.org/en/20/glossary.html#term-bound-parameters).

К примеру мы хотим отфильтровать результирующий вывод с помощью ключевого слова `WHERE`.А именно, чтобы в результате вывелись строки, в которых значение колонки `y` больше 2.

Конструкция `text()` принимает наш параметр с ведущим двоеточием `:y`, а фактическое значение параметра передается в качестве словаря, вторым аргументом в `Connection.execute()`.

В примере ниже вместо `:y` подставляется число `2`.

In [7]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 2})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-07-17 21:36:57,478 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,478 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > %(y)s
2023-07-17 21:36:57,479 INFO sqlalchemy.engine.Engine [generated in 0.00078s] {'y': 2}
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2023-07-17 21:36:57,479 INFO sqlalchemy.engine.Engine ROLLBACK


**Передача нескольких параметров.**

В примерах выше, когда мы разбирали фиксацию (коммит) транзакций, мы выполняли оператор `INSERT`, в котором, как оказалось, можно было одновременно вводить в базу данных несколько строк. Для DML-операторов, таких как `"INSERT"`, `"UPDATE"` и `"DELETE"`, мы можем передать **несколько наборов параметров** в метод `Connection.execute()`, передав список словарей вместо одного словаря, в данном случае, один SQL-оператор будет вызван несколько раз, по одному разу для каждого набора параметров. Такой стиль выполнения известен как [executemany](https://docs.sqlalchemy.org/en/20/glossary.html#term-executemany):

In [8]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}],
    )
    conn.commit()

2023-07-17 21:36:57,483 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,483 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (%(x)s, %(y)s)
2023-07-17 21:36:57,484 INFO sqlalchemy.engine.Engine [cached since 0.02015s ago] [{'x': 11, 'y': 12}, {'x': 13, 'y': 14}]
2023-07-17 21:36:57,484 INFO sqlalchemy.engine.Engine COMMIT


Ключевое различие между `"execute"` и `"executemany"` заключается в том, что последний не поддерживает возврат результриующих строк, даже если в операторе присутствует предложение `RETURNING`. Единственным исключением является использование конструкции Core [insert()](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.insert), представленной далее в этом учебном пособии в разделе ["Использование утверждений INSERT"](https://docs.sqlalchemy.org/en/20/tutorial/data_insert.html#tutorial-core-insert), которая также указывает на возврат с помощью метода [Insert.returning()](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.Insert.returning). В этом случае SQLAlchemy использует специальную логику для реорганизации оператора `INSERT` таким образом, что он может быть вызван для многих строк и при этом поддерживать `RETURNING`.

### Executing with an ORM Session

Большинство примеров вывше также применимы и к ORM части SQLAlchemy.

Здесь мы представим совместное использование Core и ORM.

Фундаментальным объектом для транзакций/подключений/интерактивности к БД при использовании ORM называется [Session](https://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session).

В современном SQLAlchemy этот объект используется также, как и `Connection`, на самом деле при использовании `Session` он внутренне ссылается на `Connection`.

Когда `Session` используется в конструкциях, не относящихся к ORM, то это фактически ничем не отличается от того, что делает `Connection`. Мы проиллюстрируем данное описание на примере ниже, в простых SQL-операциях с `text()`, которые мы изучили.

`Session` имеет много различных шаблонов создания, но здесь мы проиллюстрируем самый простой, который в точности соответствует тому, как мы бы использовали `Connection` в менеджере контекста.

In [9]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 6})
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2023-07-17 21:36:57,547 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,547 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > %(y)s ORDER BY x, y
2023-07-17 21:36:57,547 INFO sqlalchemy.engine.Engine [generated in 0.00037s] {'y': 6}
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2023-07-17 21:36:57,548 INFO sqlalchemy.engine.Engine ROLLBACK


Обратите внимение, что `Session` принимает в качестве параметра, `engine`, здесь фактически происходит тоже самое, что и при `Connection.connect()`.

Кроме прочего, `Session`, как и `Connection`, имеет возможность фиксировать данные с помощью `Session.commit()`, что показано ниже на примере SQL-оператора `UPDATE`.

In [10]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y": 11}, {"x": 13, "y": 15}],
    )
    session.commit()

2023-07-17 21:36:57,552 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,552 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=%(y)s WHERE x=%(x)s
2023-07-17 21:36:57,552 INFO sqlalchemy.engine.Engine [generated in 0.00030s] [{'y': 11, 'x': 9}, {'y': 15, 'x': 13}]
2023-07-17 21:36:57,553 INFO sqlalchemy.engine.Engine COMMIT


`Session` не удерживает объект `Connection` после завршения транзакции. Она получает новое соединение от движка каждый раз, когда ей требуется выпонить SQL-запрос.

### Работа с метаданными БД.


Центральным элементом Core и ORM, является язык SQL, который позволяет свободно строить SQL-запросы. Основой для этих запросов в связыке c SQLAlchemy являются *объекты Python*, которые  описывают таблицы и столбцы. Эти объекты в совокупности называются [метаданными БД](https://docs.sqlalchemy.org/en/20/glossary.html#term-database-metadata). 

Формально, термин *метаданные* **обычно означает данные** **которые описывают другие данные**. В SQLAlchemy термин `метаданные`, относится к конструкции [`MetaData`](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.MetaData), который представляет собой набор нформации о таблицах, столбцах, констрейнтах и других объектах DDL, которые могут описывать какую-то таблицу и существовать в БД.

Наиболее распространенными, фундаментальными объектами для метаданных БД в SQLAlchemy являются `MetaData`, `Table`, `Column`.

Ниже будет показано как эти объекты используются как в Core-ориентированном, так и ORM-ориентированном стиле.

В SQLAlchemy `таблицу` базы данных представляет/описывает объект который называется [Table](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.Table).

Перед тем как начать пользоваться `языком выраженый SQLAlchemy`, необходимо создать объекты `Table`, представляющие/описывающие таблицы БД, с которыми мы хотим работать. Объекты `Table`, создается либо с помощью `конструктора Table`, либо с помощью `ORM Mapped`. Существует также возможность загрузки информации о существующей таблице в БД, что называется [reflection](https://docs.sqlalchemy.org/en/20/glossary.html#term-reflection).

Какой бы подход ни использовался, мы всегда начинаем с коллекции, в которой будут размещаться наши таблицы, называемой объектом `MetaData`. Этот объект, по сути является оберткой/[facade](https://docs.sqlalchemy.org/en/20/glossary.html#term-facade) словаря Python,  в котором хранится ряд объектов `Table`, с ключом в виде их строкового имени. Хотя ORM предоставляет несколько вариантов, где можно получить эту коллекцию... у нас всегда есть возможность создать ее напрямую:

In [11]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

После того как у нас есть объект `MetaData`, мы можем объявить несколько объектов `Table`.

Опишем классическую модель, в которой есть таблица `user_account`, хранящая к примеру пользователей веб-сайта, и связанная с ней таблица `address`, хранящая адреса электронной почти, связанные со строками в таблице `user_account`.

Когда мы не используем `декларативные модели ORM`, то мы создаем каждый объект `Table` присваивая его переменной, которя в свою очередь будет ссылаться на таблице в коде приложения.

In [12]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
)

В приведенном выше примере, когда мы захотим обратиться к таблице `user_account` в БД, мы будем использовать Python-переменную `user_table`.

Можно заметить, что конструкция `Table`, похожа на SQL-выражение `CREATE TABLE`. Мы начинаем с имени таблицы, затем перечисляем все столбцы и их типы.

`Column` - представляет собой столбец таблицы базы данных и присваивается объекту `Table`. `Column` обычно включает в себя строковое имя и объект типа. Доступ к коллекции объектов `Column` в терминах родительской таблицы обычно осуществляется через ассоциативный массив, расположенный по адресу `Table.c`:

In [13]:
user_table.c.keys()

['id', 'name', 'fullname']

In [14]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

`Integer`, `String` - эти классы представляют типы данных SQL и могут быть переданы в `Column` как с инстанцированием, так и без него. Выше мы хотели задать столбцу "`name`" длину "30", поэтому инстанцировали `String(30)`. Но для "`id`" и "`fullname`" мы их не указывали, поэтому можем передать сам класс.

Теперь нам нужно создать вторую таблицу, которая будет иметь констрейнт `ForeignKey`:

In [15]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
)

В приведеннем выше примере также в столбце, которая является внешним ключом, мы указали `nullable=False` что эквивалентно `NOT NULL`.

При использовании объекта `ForeignKey` в определении столбца можно не указывать тип данных для этого столбца; он автоматически подставляется из типа данных связанного столбца, в приведенном выше примере - типа `Integer` столбца `user_account.id`.

В следующем разделе мы выдадим готовый DDL для таблицы пользователей и адресов, чтобы увидеть готовый результат.


Мы построили объектную структуру, представляющую две таблицы в базе данных, начиная с корневого объекта `MetaData` и заканчивая двумя объектами `Table`, каждый из которых содержит коллекцию объектов `Column` и [Constraint](https://docs.sqlalchemy.org/en/20/core/constraints.html#sqlalchemy.schema.Constraint). Эта объектная структура будет в центре большинства операций, выполняемых нами в Core и ORM в дальнейшем.

Первое, что мы можем сделать с помощью этой структуры, - это выполнить операторы `CREATE TABLE`, или DDL, для нашей базы данных, чтобы мы могли вставлять и запрашивать данные из них. 

У нас уже есть все необходимые для этого инструменты: мы вызываем метод `MetaData.create_all()` для наших `MetaData`, передавая им `Engine`, который ссылается на целевую базу данных:

In [16]:
metadata_obj.create_all(engine)

2023-07-17 21:36:57,575 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,577 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-07-17 21:36:57,577 INFO sqlalchemy.engine.Engine [generated in 0.00030s] {'table_name': 'user_account', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-07-17 21:36:57,579 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog

Приведенный выше процесс создания DDL включает несколько специфических операторов, которые проверяют существование каждой таблицы перед выполнением `CREATE`. Вся серия шагов также включена в пару `BEGIN/COMMIT`, что позволяет использовать транзакционный DDL.

Процесс создания также заботится о том, чтобы операторы `CREATE` выполнялись в правильном порядке; выше ограничение `FOREIGN KEY` зависит от существования таблицы `user`, поэтому таблица `address` создается **второй**. В более сложных сценариях зависимостей ограничения `FOREIGN KEY` могут также применяться к таблицам постфактум с помощью `ALTER`.

В объекте `MetaData` также имеется метод `MetaData.drop_all()`, который для удаления элементов схемы будет выдавать операторы `DROP` в обратном порядке, чем при выдаче `CREATE`.

Использование `декларативных форм ORM` для определения метаданных таблицы.

Другой способ создания объектов `Table`?

В предыдущих примерах было показано прямое использование объекта `Table`, который лежит в основе того, как SQLAlchemy в конечном итоге обращается к таблицам базы данных при построении SQL-выражений. Как уже упоминалось, в SQLAlchemy ORM предусмотрен фасад, позволяющий обойти процесс объявления `Table`, называемый **Declarative Table**. Процесс Declarative `Table` достигает той же цели, что и в предыдущем разделе, - создание объектов `Table`, но в рамках этого процесса дает нам нечто иное, называемое [ORM mapped class](https://docs.sqlalchemy.org/en/20/glossary.html#term-ORM-mapped-class), или просто "`mapped class`". Сопоставленный класс является наиболее распространенной основополагающей единицей SQL при использовании ORM, а в современной SQLAlchemy может также достаточно эффективно применяться и в Core-ориентированном использовании.

К числу преимуществ использования декларативной таблицы относятся:

- Более лаконичный и питонический стиль задания определений столбцов, где типы Python могут использоваться для представления типов SQL, используемых в базе данных

- Результирующий сопоставленный класс может быть использован для формирования SQL-выражений, которые во многих случаях поддерживают информацию о типизации, предусмотренную [PEP 484](https://peps.python.org/pep-0484/), которая учитывается средствами статического анализа, такими как Mypy, и средствами проверки типов IDE.

- Позволяет одновременно объявлять метаданные таблицы и ORM mapped class, используемый в операциях сохранения/загрузки объектов.

В этом разделе будет показано, как метаданные таблицы, описанные в предыдущих разделах, создаются с помощью Declarative Table.

При использовании ORM, процесс объявления метаданных таблицы совмещается с процессом объявления `ORM mapped class`.

`ORM mapped class` - это по сути любой класс Python, содержащий атрибуты связанные колонками таблицы БД.

Хотя существует несколько видов этого процесса, наиболее рапространенный стиль известен как **декларативный**, что позволяет нам одновременно объявлять пользовательские классы и метаданные таблцы.

Создание декларативной базы.

При использовании ORM, коллекция `MetaData` сохраняется, однако сама она ассоциируется с конструкцией предназначенной только для ORM и называемой декларативной базой (Declarative Base). Наиболее целесообразным способом получения новой декларативной базы является создание нового класса, который является подклассом класса Declarative Base.

In [17]:
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

Выше мы создали класс `Base`, который мы будем называть `Declarative Base`. Когда мы создаем новые классы являющиеся подклассом `Base`, каждый из них будет создат как новый `маппед класс ORM`, каждый из которых обычно ссылается на определенный объект `Table`.

Декларативная база ссылается на коллекцию `MetaData`, которая создается для нас автоматически, если мы не предоставили ее извне. Доступ к этой коллекции `MetaData` осуществляется через атрибут уровня класса [DeclarativeBase.metadata](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase.metadata). При создании новых отображаемых классов каждый из них будет ссылаться на таблицу в этой коллекции `MetaData`:

In [18]:
Base.metadata

MetaData()

Декларативная база также ссылается на коллекцию под названием [registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry), которая является центральной единицей "`mapper configuration`" в SQLAlchemy ORM. Хотя к этому объекту редко обращаются напрямую, он занимает центральное место в процессе `mapper configuration`, поскольку набор отображаемых классов ORM будет координироваться друг с другом через этот [registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry). Как и в случае с `MetaData`, наша декларативная база также создала для нас реестр (опять же с возможностью передачи собственного [реестра](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry)), доступ к которому мы можем получить через переменную класса [DeclarativeBase.registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase.registry):

In [19]:
Base.registry

[DeclarativeBase](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase) - это не единственный способ отображения классов, а только самый распространенный. В [registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry) представлены и другие конфигурационные паттерны отображения, включая декораторно-ориентированный и императивный способы отображения классов. Также имеется полная поддержка создания Python-классов данных при отображении. Все это можно найти в справочной документации по конфигурации [отображаемых классов в ORM](https://docs.sqlalchemy.org/en/20/orm/mapper_config.html).



Объявление `mapped classes`/`отображаемых классов`/`классов сопоставления`/`маппируемых классов`.

Теперь, когда класс `Base` создан, мы можем определить `ORM-маппируемые классы` для таблиц `user_account` и `address` в терминах новых классов `User` и `Address`. Ниже мы проиллюстрируем наиболее современную форму `Declarative`, которая основана на аннотациях типов [PEP 484](https://peps.python.org/pep-0484/) с использованием специального типа `Mapped`, указывающего на атрибуты, которые должны быть отображены как определенные типы:

In [20]:
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

Два класса выше, `User` и `Address`, теперь называются `ORM Mapped Classes` и доступны для использования в операциях персистентного хранения и запросов ORM, которые будут описаны позже. 

Более подробная информация об этих классах включает:

- Каждый класс ссылается на объект `Table`, сгенерированный в процессе декларативного отображения/маппинга, который именуется путем присвоения строки атрибуту `DeclarativeBase.__tablename__`. После создания класса эта сгенерированная таблица становится доступной по атрибуту `DeclarativeBase.__table__`.

- Как упоминалось, такая форма называется [декларативной конфигурацией таблиц](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-table-configuration). Мы создаем объект `Table` напрямую и присваиваем его непосредственно к `DeclarativeBase.__table__`. Этот стиль известен как `Declarative with Imperative Table`.

- Для указания столбцов в таблице используется конструкция [mapped_column()](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.mapped_column) в сочетании с аннотациями типов, основанными на типе [Mapped](https://docs.sqlalchemy.org/en/20/orm/internals.html#sqlalchemy.orm.Mapped). Этот объект будет генерировать объекты `Column`, которые применяются при построении `Table`.

- Для столбцов с простыми типами данных, не имеющих других вариантов, мы можем указать только аннотацию `Mapped`, используя такие простые типы Python, как `int` и `str`, для обозначения `Integer` и `String`. Настройки интерпретации типов Python в процессе декларативного отображения очень свободны; см. разделы [Использование аннотированной декларативной таблицы (Формы с аннотацией типов для mapped_column())](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-mapped-column) и [Настройка Типов Map](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-mapped-column-type-map).

- Колонка может быть объявлена как "nullable" или "not null" на основании наличия аннотации типа `Optional[<typ>]` (или ее эквивалентов `<typ> | None или Union[<typ>, None]`). Параметр [mapped_column.nullable](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.mapped_column.params.nullable) также может быть использован явно (и не обязательно должен соответствовать опциональности аннотации).

- Использование явных аннотаций типов совершенно необязательно. Мы также можем использовать `mapped_column()` без аннотаций. В этом случае в каждой конструкции `mapped_column()` следует использовать более явные объекты типа `Integer` и `String`, а также `nullable=False` по мере необходимости.

- Два дополнительных атрибута, `User.addresses` и `Address.user`, определяют другой тип атрибута, называемый [relationship()](https://docs.sqlalchemy.org/en/20/orm/relationship_api.html#sqlalchemy.orm.relationship), который имеет аналогичные аннотационно-ориентированные настройки типов, как показано в примере. Более подробно конструкция [relationship()](https://docs.sqlalchemy.org/en/20/orm/relationship_api.html#sqlalchemy.orm.relationship) рассматривается в разделе [Работа со связанными объектами ORM](https://docs.sqlalchemy.org/en/20/tutorial/orm_related_objects.html#tutorial-orm-related-objects).

- Классы автоматически получают метод `__init__()`, если мы не объявляем его самостоятельно. В стандартной форме этот метод принимает все имена атрибутов в качестве необязательных аргументов-ключей:
```python
sandy = User(name="sandy", fullname="Sandy Cheeks")
```

- Для автоматической генерации полнофункционального метода `__init__()`, в котором предусмотрены позиционные аргументы, а также аргументы со значениями ключевых слов по умолчанию, можно использовать функцию dataclasses, представленную в разделе [Declarative Dataclass Mapping](https://docs.sqlalchemy.org/en/20/orm/dataclasses.html#orm-declarative-native-dataclasses). Конечно, всегда можно использовать и явный метод `__init__()`.

- Методы `__repr__()` добавлены для того, чтобы мы получили читаемый строковый вывод; наличие этих методов не является обязательным. Как и в случае с `__init__()`, метод `__repr__()` может быть сгенерирован автоматически с помощью функции [dataclasses](https://docs.sqlalchemy.org/en/20/orm/dataclasses.html#orm-declarative-native-dataclasses).


Куда делась старая добрая декларативность?

Пользователи `SQLAlchemy 1.4` и более ранних версий заметят, что приведенное выше отображение/mapping имеет принципиально иную форму, чем раньше; оно не только использует `mapped_column()` вместо `Column` в декларативном отображении/mapping, но и использует аннотации типов Python для получения информации о столбцах.

Для пользователей "старого" способа поясним, что декларативные отображения/mapping по-прежнему могут выполняться с использованием объектов `Column` (а также с использованием функции [declarative_base()](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.declarative_base) для создания базового класса), и эти формы будут поддерживаться и в дальнейшем, причем их отмена не планируется. Причина замены этих двух объектов новыми конструкциями заключается, прежде всего, в том, что они легко интегрируются с инструментами `PEP 484`, включая `IDE`, такие как `VSCode`, и программы проверки типов, такие как `Mypy` и `Pyright`, без необходимости установки плагинов. Во-вторых, вывод деклараций из аннотаций типов является частью интеграции SQLAlchemy с датаклассами Python, которые теперь могут быть [сгенерированы нативно](https://docs.sqlalchemy.org/en/20/orm/dataclasses.html#orm-declarative-native-dataclasses) из отображений/mappings.

Для пользователей, которым нравится "старый" способ, но при этом они хотят, чтобы их IDE не выдавала ошибочных сообщений об ошибках ввода в декларативных отображениях/mappings, конструкция `mapped_column()` является полноценной заменой `Column` в декларативных отображениях ORM (обратите внимание, что `mapped_column()` предназначена только для декларативных отображений ORM; она не может использоваться внутри конструкции `Table`), а аннотации типов являются необязательными. Наше отображение, приведенное выше, можно записать без аннотаций как:

```python
class User(Base):
    __tablename__ = "user_account"

    id = mapped_column(Integer, primary_key=True)
    name = mapped_column(String(30), nullable=False)
    fullname = mapped_column(String)

    addresses = relationship("Address", back_populates="user")

    # ... продолжение опеределений
```

См. также:

- [ORM Mapping Styles](https://docs.sqlalchemy.org/en/20/orm/mapping_styles.html#orm-mapping-styles) -  полная информация о различных конфигурационных стилях ORM.
- 
- [https://docs.sqlalchemy.org/en/20/orm/mapping_styles.html#orm-declarative-mapping](https://docs.sqlalchemy.org/en/20/orm/mapping_styles.html#orm-declarative-mapping) - обзор декларативного отображения классов


- [https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-table](https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-table) - подробно описано использование функций `mapped_column()` и `Mapped` для определения столбцов `Table`, которые должны быть сопоставлены при использовании Declarative.


Генерация DDL в базу данных из `ORM mapping`.

Поскольку наши классы, отображаемые в ORM, ссылаются на объекты `Table`, содержащиеся в коллекции `MetaData`, генерация DDL на основе декларативной базы происходит аналогично процессу, описанному ранее в разделе. В нашем случае мы уже создавали таблицы `пользователей` и `адресов` в нашей базе данных выше. Если бы мы этого еще не сделали, то могли бы воспользоваться для этого метаданными, связанными с нашим классом ORM Declarative Base, получив доступ к коллекции из атрибута [DeclarativeBase.metadata](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase.metadata), а затем, как и раньше, использовать [MetaData.create_all()](https://docs.sqlalchemy.org/en/20/core/metadata.html#sqlalchemy.schema.MetaData.create_all). В этом случае выполняются операторы *(IF EXISTS)*, но новые таблицы не создаются, так как они уже присутствуют:

In [21]:
Base.metadata.create_all(engine)

2023-07-17 21:36:57,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,608 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-07-17 21:36:57,609 INFO sqlalchemy.engine.Engine [cached since 0.03163s ago] {'table_name': 'user_account', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-07-17 21:36:57,610 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_cat

Далее мы рассмотрим, как мы можем генерировать объекты `Table` из существующей БД.

Завершая раздел о работе с метаданными таблиц, мы проиллюстрируем еще одну операцию, о которой говорилось в начале раздела, - отражение таблицы (**table reflection**). Под отражением таблицы (**table reflaction**) понимается процесс генерации объектов `Table` и связанных с ними объектов путем чтения текущего состояния базы данных. Если в предыдущих разделах мы объявляли объекты `Table` в Python, после чего у нас была возможность отправить DDL в базу данных для создания схемы, то в процессе отражения эти два шага выполняются в обратном порядке, начиная с существующей базы данных и создавая структуры данных на языке Python для представления схем в этой базе.


В качестве примера отражения (reflection) создадим новый объект `Table`, представляющий объект `some_table`, который мы создали вручную в предыдущих разделах этого документа. Существует несколько вариантов того, как это делается, однако наиболее простой заключается в создании объекта `Table` с указанием имени таблицы и коллекции `MetaData`, к которой он будет принадлежать, а затем вместо указания отдельных объектов `Column` и `Constraint` передать ему целевой `Engine` с помощью параметра `Table.autoload_with`:

In [22]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2023-07-17 21:36:57,616 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,617 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_attribute.attname AS name, pg_catalog.format_type(pg_catalog.pg_attribute.atttypid, pg_catalog.pg_attribute.atttypmod) AS format_type, (SELECT pg_catalog.pg_get_expr(pg_catalog.pg_attrdef.adbin, pg_catalog.pg_attrdef.adrelid) AS pg_get_expr_1 
FROM pg_catalog.pg_attrdef 
WHERE pg_catalog.pg_attrdef.adrelid = pg_catalog.pg_attribute.attrelid AND pg_catalog.pg_attrdef.adnum = pg_catalog.pg_attribute.attnum AND pg_catalog.pg_attribute.atthasdef) AS "default", pg_catalog.pg_attribute.attnotnull AS not_null, pg_catalog.pg_class.relname AS table_name, pg_catalog.pg_description.description AS comment, pg_catalog.pg_attribute.attgenerated AS generated, (SELECT json_build_object(%(json_build_object_2)s, pg_catalog.pg_attribute.attidentity = %(attidentity_1)s, %(json_build_object_3)s, pg_catalog.pg_sequence.seqstart, %(json_build_object_4)s, pg_catalog

В конце процесса объект `some_table` теперь содержит информацию об объектах `Column`, присутствующих в таблице, и может использоваться точно так же, как и таблица, которую мы объявили явно:

In [23]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

Теперь у нас есть готовая база данных с двумя таблицами, а также таблично-ориентированные конструкции Core и ORM, которые мы можем использовать для взаимодействия с этими таблицами через `Connection` и/или `ORM Session`. В следующих разделах мы проиллюстрируем, как создавать, манипулировать и выбирать данные с помощью этих конструкций.

Декларативная база также ссылается на коллекцию под названием [registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry), которая является центральной единицей "`mapper configuration`" в SQLAlchemy ORM. Хотя к этому объекту редко обращаются напрямую, он занимает центральное место в процессе `mapper configuration`, поскольку набор отображаемых классов ORM будет координироваться друг с другом через этот [registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry). Как и в случае с `MetaData`, наша декларативная база также создала для нас реестр (опять же с возможностью передачи собственного [реестра](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.registry)), доступ к которому мы можем получить через переменную класса [DeclarativeBase.registry](https://docs.sqlalchemy.org/en/20/orm/mapping_api.html#sqlalchemy.orm.DeclarativeBase.registry):

Использование `INSERT-выражений`.

При использовании Core и ORM, `SQL INSERT` формируется с помощью функции [insert()](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.insert) - эта функция генерирует новый экземпляр `Insert`, который представляет собой оператор `INSERT` в SQL, добавляющий новые данные в таблицу.

In [24]:
from sqlalchemy import insert
stmt = insert(user_table).values(name="spongebob", fullname="Spongebob Squarepants")

# smts - является инстансом Insert

В примере выше, `stmt` - это инстанс [Insert](https://docs.sqlalchemy.org/en/20/core/dml.html#sqlalchemy.sql.expression.Insert).

Мы также можем передать функции принт эту переменную, и получить строчное представление SQL-запроса:

In [25]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


Строчная форма создается путем создания компилированный формы объекта, которая включает в себя специфическое для базы даннх SQL-представление оператора. Мы можем получить этот объект непосредственно с помощью метода `ClauseElemnt.compile()`

In [26]:
compiled = stmt.compile()

In [27]:
compiled.params

{'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}

Теперь мы можем выполнить это выражение.

In [28]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2023-07-17 21:36:57,666 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,666 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s) RETURNING user_account.id
2023-07-17 21:36:57,667 INFO sqlalchemy.engine.Engine [generated in 0.00113s] {'name': 'spongebob', 'fullname': 'Spongebob Squarepants'}
2023-07-17 21:36:57,668 INFO sqlalchemy.engine.Engine COMMIT


В своей простой форме оператор `INSERT` не возвращает никаких строк, а если вставляется только одна строка, то он обычно включает в себя возможность возврата информации о значениях по умолчанию на уровне столбцов, которые были сгенерированы во время `INSERT` этой строки, чаще всего целочисленного значения первичного ключа. В приведенном выше случае первая строка в базе данных обычно возвращает 1 для первого целочисленного значения первичного ключа, которое мы можем получить с помощью аксессора [CursorResult.inserted_primary_key](https://docs.sqlalchemy.org/en/20/core/connections.html#sqlalchemy.engine.CursorResult.inserted_primary_key):

In [29]:
result.inserted_primary_key  # (1,)

(1,)

In [30]:
another_stmt = insert(user_table).values(name="skwidward", fullname="Skwidward")
with engine.connect() as conn:
    result = conn.execute(another_stmt)
    conn.commit()
result.inserted_primary_key  # (2,)

2023-07-17 21:36:57,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,676 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name)s, %(fullname)s) RETURNING user_account.id
2023-07-17 21:36:57,676 INFO sqlalchemy.engine.Engine [cached since 0.01029s ago] {'name': 'skwidward', 'fullname': 'Skwidward'}
2023-07-17 21:36:57,677 INFO sqlalchemy.engine.Engine COMMIT


(2,)

В приведенном примере метод `Insert.values()` использовался для явного создания пункта `VALUES` оператора `SQL INSERT`. Если мы не используем метод `Insert.values()`, а просто выведем "пустой" оператор, то получим `INSERT` для каждого столбца таблицы:

In [31]:
print(insert(user_table))

INSERT INTO user_account (id, name, fullname) VALUES (:id, :name, :fullname)


Если взять конструкцию `Insert`, для которой не вызывалась функция `Insert.values()`, и выполнить ее, а не вывести на печать, то оператор будет скомпилирован в строку на основе параметров, переданных методу `Connection.execute()`, и будет включать только столбцы, соответствующие переданным параметрам. Собственно, это и есть обычный способ использования `Insert` для вставки строк без необходимости вводить явное предложение `VALUES`. Приведенный ниже пример иллюстрирует выполнение оператора `INSERT` с двумя столбцами и сразу со списком параметров:

In [32]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "sandy", "fullname": "Sandy Cheeks"},
            {"name": "patrick", "fullname": "Patrick Star"},
        ],
    )
    conn.commit()

2023-07-17 21:36:57,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,685 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (%(name__0)s, %(fullname__0)s), (%(name__1)s, %(fullname__1)s)
2023-07-17 21:36:57,685 INFO sqlalchemy.engine.Engine [generated in 0.00051s (insertmanyvalues) 1/1 (unordered)] {'name__0': 'sandy', 'fullname__0': 'Sandy Cheeks', 'name__1': 'patrick', 'fullname__1': 'Patrick Star'}
2023-07-17 21:36:57,686 INFO sqlalchemy.engine.Engine COMMIT


Приведенное выше выполнение имеет форму "`executemany`", впервые показанную в разделе ["Передача нескольких параметров"](https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#tutorial-multiple-parameters), однако, в отличие от использования конструкции `text()`, нам не пришлось прописывать никакой SQL. Передавая словарь или список словарей в метод `Connection.execute()` в сочетании с конструкцией `Insert`, `Connection` гарантирует, что переданные имена столбцов будут автоматически выражены в предложении `VALUES` конструкции `Insert`.

Ниже приведен более продвинутый пример, иллюстрирующий, как можно использовать метод `Insert.values()` в явном виде и в то же время включать дополнительные `VALUES`, генерируемые из параметров. Строится [скалярный подзапрос](https://docs.sqlalchemy.org/en/20/glossary.html#term-scalar-subquery) с использованием конструкции [select()](https://docs.sqlalchemy.org/en/20/core/selectable.html#sqlalchemy.sql.expression.select), представленной в следующем разделе, а параметры, используемые в подзапросе, задаются с помощью явного связанного имени параметра, устанавливаемого с помощью конструкции `bindparam()`.

Это несколько более глубокая алхимия для того, чтобы мы могли добавлять связанные строки без извлечения идентификаторов первичных ключей из операции `user_table` в приложение. Большинство алхимиков просто используют ORM, который позаботится о подобных вещах за нас.

In [33]:
from sqlalchemy import select, bindparam
scalar_subq = (
    select(user_table.c.id)
    .where(user_table.c.name == bindparam("username"))
    .scalar_subquery()
)

with engine.connect() as conn:
    result = conn.execute(
        insert(address_table).values(user_id=scalar_subq),
        [
            {
                "username": "spongebob",
                "email_address": "spongebob@sqlalchemy.org",
            },
            {"username": "sandy", "email_address": "sandy@sqlalchemy.org"},
            {"username": "sandy", "email_address": "sandy@squirrelpower.org"},
        ],
    )
    conn.commit()

2023-07-17 21:36:57,690 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-17 21:36:57,691 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = %(username__0)s), %(email_address__0)s), ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = %(username__1)s), %(email_address__1)s), ((SELECT user_account.id 
FROM user_account 
WHERE user_account.name = %(username__2)s), %(email_address__2)s)
2023-07-17 21:36:57,691 INFO sqlalchemy.engine.Engine [generated in 0.00053s (insertmanyvalues) 1/1 (unordered)] {'email_address__0': 'spongebob@sqlalchemy.org', 'username__0': 'spongebob', 'email_address__1': 'sandy@sqlalchemy.org', 'username__1': 'sandy', 'email_address__2': 'sandy@squirrelpower.org', 'username__2': 'sandy'}
2023-07-17 21:36:57,693 INFO sqlalchemy.engine.Engine COMMIT


`INSERT...RETURNING`

Предложение `RETURNING` для поддерживаемых *бэкендов* используется автоматически для получения последнего вставленного значения первичного ключа, а также значений для умолчаний сервера. Однако условие `RETURNING` может быть задано и явно с помощью метода `Insert.returning()`; в этом случае объект `Result`, возвращаемый при выполнении оператора, содержит строки, которые могут быть извлечены:

In [34]:
insert_stmt = insert(address_table).returning(
    address_table.c.id, address_table.c.email_address
)
print(insert_stmt)

'''
INSERT INTO address (id, user_id, email_address)
VALUES (:id, :user_id, :email_address)
RETURNING address.id, address.email_address
'''

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


'\nINSERT INTO address (id, user_id, email_address)\nVALUES (:id, :user_id, :email_address)\nRETURNING address.id, address.email_address\n'

Его также можно комбинировать с `Insert.from_select()`, как в приведенном ниже примере, развивающем пример, изложенный в `INSERT...FROM SELECT`:

In [35]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address",], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


`INSERT...FROM SELECT`

Менее используемая возможность `Insert`, но приведенная здесь для полноты картины, конструкция `Insert` может составлять `INSERT`, получающий строки непосредственно из `SELECT` с помощью метода `Insert.from_select()`. Этот метод принимает конструкцию `select()`, которая рассматривается в следующем разделе, а также список имен столбцов, на которые будет ориентирована `INSERT`. В приведенном ниже примере в таблицу `address` добавляются строки, полученные из строк таблицы `user_account`, в результате чего каждый пользователь получает бесплатный адрес электронной почты на сайте aol.com:

In [37]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


Эта конструкция используется в тех случаях, когда требуется скопировать данные из другой части базы данных непосредственно в новый набор строк, без необходимости получения и повторной отправки данных от клиента.